# Decoding a neural network from a JSON file to Python, then encoding the hessian from laplace.py to a JSON

In [1]:
# imports
from laplace import Laplace

import numpy as np
import pandas as pd
import torch

import json

import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

torch.manual_seed(43)
torch.set_printoptions(sci_mode=False)

In [2]:
# Import data from csv

# Load data from CSV file using pandas
df = pd.read_csv('data_multi.csv')

# Split the dataframe into x and y tensors
x = torch.from_numpy(df[['x1', 'x2']].to_numpy()).to(torch.float32)
y = torch.from_numpy(df['y'].to_numpy(dtype=int))

X = x.T

y_unique = torch.unique(y)
y_indices = y - 1
y_indices = y_indices.long()
y_train = nn.functional.one_hot(y_indices, num_classes=len(y_unique)).float()
y_train

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1.,

In [3]:
# Init model

data = list(zip(x, y_train))
n_hidden = 3
D = X.shape[0]  # == 2
out_dim = y_train.shape[1]  # == 4
model = nn.Sequential(
    nn.Linear(D, n_hidden),
    nn.Sigmoid(),
    nn.Linear(n_hidden, out_dim)
)

loss_fn = nn.CrossEntropyLoss()

In [4]:
with open('nn_multi.json') as fin:
    nn_json_str = fin.read()
    nn_json = json.loads(nn_json_str)

nn_json

[{'bias': [-2.0739872, 1.6585876, -0.7064312],
  'weight': [[-4.0660324, 0.29628262, 0.3625705],
   [0.03851904, 1.4500277, -2.256552]]},
 {'bias': [1.3787575, -2.5149424, 0.47881675, -2.0565474],
  'weight': [[-8.137701, 6.033078, -9.181937, 5.5667276],
   [2.4892216, -8.89387, -5.512849, -0.49369964],
   [-7.280996, -0.25884467, 4.017399, -8.365101]]}]

In [5]:
assert len(model.state_dict()) == 2 * len(nn_json)
iter_states = iter(model.state_dict())

# for layer in model.state_dict():
#     print(layer)
for layer_read in nn_json:
    state_w = next(iter_states)
    state_b = next(iter_states)
    tensor_w = torch.tensor(layer_read['weight']).T
    tensor_b = torch.tensor(layer_read['bias']).T
    model.state_dict()[state_w].data.copy_(tensor_w)
    model.state_dict()[state_b].data.copy_(tensor_b)

C:\Users\adeli\AppData\Local\Temp\ipykernel_4916\3361615766.py:10: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3281.)
  tensor_b = torch.tensor(layer_read['bias']).T


In [6]:
for val in model.state_dict().values():
    display(val)

tensor([[-4.0660,  0.0385],
        [ 0.2963,  1.4500],
        [ 0.3626, -2.2566]])

tensor([-2.0740,  1.6586, -0.7064])

tensor([[-8.1377,  2.4892, -7.2810],
        [ 6.0331, -8.8939, -0.2588],
        [-9.1819, -5.5128,  4.0174],
        [ 5.5667, -0.4937, -8.3651]])

tensor([ 1.3788, -2.5149,  0.4788, -2.0565])

In [7]:
y_hat = torch.argmax(torch.softmax(model.forward(x), dim=1), dim=1) + 1

y_hat == y

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, Tr

In [8]:
print(model(x))

tensor([[  3.8487, -11.4021,  -5.0200,  -2.5696],
        [  3.8476, -11.4050,  -5.0207,  -2.5719],
        [  3.8174, -11.3840,  -4.9939,  -2.5984],
        [  3.8586, -11.3980,  -5.0351,  -2.5483],
        [  3.5138, -11.3866,  -4.8226,  -2.9442],
        [  3.5241, -11.2808,  -4.7814,  -2.8916],
        [  3.8641, -11.4066,  -5.0308,  -2.5538],
        [  3.8285, -11.3935,  -5.0046,  -2.5893],
        [  3.8600, -11.4020,  -5.0417,  -2.5452],
        [  2.6337, -11.3735,  -4.3165,  -3.9387],
        [  3.8672, -11.4083,  -5.0334,  -2.5509],
        [  3.8631, -11.4032,  -5.0290,  -2.5535],
        [  3.8552, -11.3950,  -5.0225,  -2.5580],
        [  3.8673, -11.4084,  -5.0334,  -2.5509],
        [  3.7627, -11.3534,  -4.9498,  -2.6483],
        [  3.8674, -11.4081,  -5.0333,  -2.5507],
        [  3.8672, -11.4084,  -5.0334,  -2.5510],
        [  2.9519, -11.2762,  -4.4525,  -3.5411],
        [  3.8628, -11.4058,  -5.0297,  -2.5550],
        [  3.8302, -11.3968,  -5.0070,  -2.5887],


In [15]:
la = Laplace(model, 'classification',
             subset_of_weights='all',
             hessian_structure='full')

la.fit(DataLoader(TensorDataset(x, y_train), batch_size=1))

hessian = la.H
torch.set_printoptions(precision=4, sci_mode=True)
print(hessian)
array = hessian.numpy()
np.savetxt('hessian_multi_all_full_ggn.csv', array, delimiter=',')

tensor([[ 2.7072e-06,  1.6723e-06, -5.5242e-06,  7.7286e-07, -1.4560e-06,
         -1.5128e-06,  2.0981e-06, -1.4059e-06, -6.9988e-07,  1.8376e-05,
         -1.1840e-04, -5.1769e-07, -1.4401e-05, -3.7332e-08,  2.3887e-05,
          1.3789e-05, -1.1470e-06, -2.4407e-05, -1.7764e-05,  1.1959e-04,
          1.0379e-06, -1.1858e-04,  2.3653e-05, -2.5354e-05,  1.2028e-04],
        [ 1.6723e-06,  3.1724e-05,  7.7286e-07, -1.9813e-05, -1.5128e-06,
         -2.1345e-06,  9.2902e-07,  7.5741e-07, -8.6206e-07, -2.4699e-05,
         -3.5507e-04, -1.3379e-06, -2.2362e-05, -5.9598e-07, -1.9375e-04,
          2.2075e-05, -1.2950e-06,  1.9378e-04,  2.4987e-05,  3.5696e-04,
          1.3093e-06, -3.5640e-04, -1.9343e-04,  1.9157e-04,  3.5826e-04],
        [-5.5242e-06,  7.7286e-07,  1.7039e-01, -7.7296e-02, -8.0129e-03,
         -1.3081e-03, -1.4059e-06,  8.7116e-03, -3.2322e-04, -8.0921e-03,
         -1.2578e-03,  1.2096e-02,  2.3431e-02,  1.5130e-02, -4.9810e-03,
         -1.4786e-03, -3.1149e-03, -

In [16]:
la = Laplace(model, 'classification',
             subset_of_weights='last_layer',
             hessian_structure='full')

la.fit(DataLoader(TensorDataset(x, y_train), batch_size=1))

hessian = la.H
array = hessian.numpy()
np.savetxt('hessian_multi_ll_full_ggn.csv', array, delimiter=',')

In [17]:
la = Laplace(model, 'classification',
             subset_of_weights='subnetwork',
             subnetwork_indices = torch.LongTensor([1, 3, 5, 6, 7, 9]),
             hessian_structure='full')

la.fit(DataLoader(TensorDataset(x, y_train), batch_size=1))

hessian = la.H
array = hessian.numpy()
np.savetxt('hessian_multi_subnet_full_ggn.csv', array, delimiter=',')

In [19]:
from laplace.curvature import AsdlEF
la = Laplace(model, 'classification',
             subset_of_weights='all',
             hessian_structure='full', backend=AsdlEF)

la.fit(DataLoader(TensorDataset(x, y_train), batch_size=1))

hessian = la.H
torch.set_printoptions(precision=4, sci_mode=True)
array = hessian.numpy()
np.savetxt('hessian_multi_all_full_empfisher.csv', array, delimiter=',')

In [20]:
from laplace.curvature import AsdlEF
la = Laplace(model, 'classification',
             subset_of_weights='last_layer',
             hessian_structure='full', backend=AsdlEF)

la.fit(DataLoader(TensorDataset(x, y_train), batch_size=1))

hessian = la.H
array = hessian.numpy()
np.savetxt('hessian_multi_ll_full_empfisher.csv', array, delimiter=',')

In [21]:
from laplace.curvature import AsdlEF
la = Laplace(model, 'classification',
             subset_of_weights='subnetwork',
             subnetwork_indices = torch.LongTensor([1, 3, 5, 6, 7, 9]),
             hessian_structure='full', backend=AsdlEF)

la.fit(DataLoader(TensorDataset(x, y_train), batch_size=1))

hessian = la.H
array = hessian.numpy()
np.savetxt('hessian_multi_subnet_full_empfisher.csv', array, delimiter=',')